In [2]:
import pandas as pd
import sqlite3


import sqlite3
import pandas as pd


db_path=r'/Users/admin/dev/algobetting/infra/data/db/fotmob.db'

conn = sqlite3.connect(db_path)
    
    # Load match data
pen_df = pd.read_sql_query(f"""
        SELECT 
            match_id,
            match_date,
            league_id,
            home.team_name as home_team,
            home_pens,
            away.team_name as away_team,
            away_pens
        FROM penalties fmd
            LEFT JOIN team_id_mapping home ON home.team_id = fmd.home_team
            LEFT JOIN team_id_mapping away ON away.team_id = fmd.away_team
        --WHERE
        --    league_id IN ('Premier_League')
            --AND season = ?
        ORDER BY match_date ASC
    """, conn)

pen_df

,match_id,match_date,league_id,home_team,home_pens,away_team,away_pens
0,3609929,2021-08-13,Premier_League,Brentford,0,Arsenal,0
1,3609936,2021-08-14,Premier_League,Norwich,0,Liverpool,0
2,3609930,2021-08-14,Premier_League,Burnley,0,Brighton,0
3,3609931,2021-08-14,Premier_League,Chelsea,0,Crystal Palace,0
4,3609932,2021-08-14,Premier_League,Everton,0,Southampton,0
...,...,...,...,...,...,...,...
1345,4813580,2026-01-07,Premier_League,Everton,0,Wolves,0
1346,4813583,2026-01-07,Premier_League,Newcastle,1,Leeds,1
1347,4813577,2026-01-07,Premier_League,Brentford,0,Sunderland,1
1348,4813582,2026-01-07,Premier_League,Man City,1,Brighton,0


In [9]:
# Baseline model
pen_df['home_pen_pred'] = 0.16074074074074074
pen_df['away_pen_pred'] = 0.10888888888888888

pen_df['home_abs_error'] = abs(pen_df['home_pens'] - pen_df['home_pen_pred'])
pen_df['away_abs_error'] = abs(pen_df['away_pens'] - pen_df['away_pen_pred'])

pen_df['match_pens'] = pen_df['home_pens'] + pen_df['away_pens']
pen_df['match_abs_error'] = abs(pen_df['match_pens'] - (pen_df['away_pen_pred'] + pen_df['home_pen_pred']))


print(f'Home Pens MAE: {pen_df['home_abs_error'].mean()}')
print(f'Away Pens MAE: {pen_df['away_abs_error'].mean()}')
print(f'Match Pens MAE: {pen_df['match_abs_error'].mean()}')

Home Pens MAE: 0.27290205761316877
Away Pens MAE: 0.19551604938271605
Match Pens MAE: 0.4114348422496571
